[Caustics](https://en.wikipedia.org/wiki/Caustic_(optics)) are luminous patterns which are resulting from the smooth deviation of light rays. It is the heart-shaped pattern in your cup of coffee which is formed by reflection of rays. It is also the wiggly patterns of light that you will see on the floor of a pool. Here we will simulate this physical phenomenon. It is indeed of interest as speaks to how images are formed (more on this later), hence how the brain may understand images, but also because they are mesmerizingly beautiful. In this post, I will develop a simple formalism to generate such patterns, with the paradoxical result that it is very simple to code yet generates patterns with great complexity, such as: 
<BR>
<center><img src="https://laurentperrinet.github.io/sciblog/files/2020-06-19_caustique/2020-06-19_caustique.gif" width=61.8%/> </center>
<BR>

This is joint work with artist [Etienne Rey](https://laurentperrinet.github.io/authors/etienne-rey/), in which I especially follow the ideas put forward in the series [Turbulence](http://ondesparalleles.org/projets/turbulences/).

<!-- TEASER_END -->

Let's begin with an import of the library that we will need: the essential [numpy](https://numpy.org/) and [matplotlib](https://matplotlib.org/) but also [MotionClouds](https://github.com/NeuralEnsemble/MotionClouds) to generate a simple waveform (more on this later in this post).

In [1]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import MotionClouds as mc

Let's define an unique identifier

In [2]:
import datetime
date = datetime.datetime.now().date().isoformat() # https://en.wikipedia.org/wiki/ISO_8601
tag = f'{date}_caustique'
tag = '2020-06-19_caustique'
print(f'Tagging our simulations with tag={tag}')

Tagging our simulations with tag=2020-06-19_caustique


And a `Namespace` object to store all parameters:

In [3]:
def init(args=[]):
    import argparse

    parser = argparse.ArgumentParser()
    parser.add_argument("--tag", type=str, default='caustique', help="Tag")
    parser.add_argument("--nx", type=int, default=5*2**8, help="number of pixels (vertical)")
    parser.add_argument("--ny", type=int, default=8*2**8, help="number of pixels (horizontal)")
    parser.add_argument("--bin_dens", type=int, default=4, help="relative bin density")
    parser.add_argument("--nframe", type=int, default=2**7, help="number of frames")
    parser.add_argument("--seed", type=int, default=42, help="seed for RNG")
    parser.add_argument("--H", type=float, default=80., help="depth")
    parser.add_argument("--sf_0", type=float, default=0.002, help="sf")
    parser.add_argument("--B_sf", type=float, default=0.001, help="bandwidth in sf")
    parser.add_argument("--V_Y", type=float, default=0.5, help="horizontal speed")
    parser.add_argument("--V_X", type=float, default=0.5, help="vertical speed")
    parser.add_argument("--B_V", type=float, default=1.0, help="bandwidth in speed")
    parser.add_argument("--theta", type=float, default=np.pi/2, help="angle with the horizontal")
    parser.add_argument("--B_theta", type=float, default=np.pi/12, help="bandwidth in theta")
    parser.add_argument("--fps", type=float, default=18, help="bandwidth in theta")
    parser.add_argument("--verbose", type=bool, default=False, help="Displays more verbose output.")

    opt = parser.parse_args(args=args)

    if opt.verbose:
        print(opt)
    return opt
opt = init()
opt.tag = tag
figpath=f'../files/{opt.tag}'

In [4]:
opt

Namespace(B_V=1.0, B_sf=0.001, B_theta=0.2617993877991494, H=80.0, V_X=0.5, V_Y=0.5, bin_dens=4, fps=18, nframe=128, nx=1280, ny=2048, seed=42, sf_0=0.002, tag='2020-06-19_caustique', theta=1.5707963267948966, verbose=False)

This object is always handy when we will try to explore some parameters.

A last utility function is a way to convert a numpy array to an animated GIF using [imageio](https://imageio.github.io/) and [pygifsicle](https://github.com/LucaCappelletti94/pygifsicle):

In [5]:
def make_gif(gifname, fnames, fps):
    import imageio

    with imageio.get_writer(gifname, mode='I', fps=fps) as writer:
        for fname in fnames:
            writer.append_data(imageio.imread(fname))

    from pygifsicle import optimize
    optimize(str(gifname))
    return gifname

from IPython.display import Image, display
width = 1024

# What is a caustic?

A caustic usually refers to the image which is formed by the convergence or divergence of rays on a surface. Mathematically, this may be understood as the enveloppe of a set of rays, and the best known example are that formed when the sun hits a glass or cup and you see heart-shaped pattern (actually more a kidney than a heart, hence the name "nephroid"):

In [6]:
display(Image(url='https://upload.wikimedia.org/wikipedia/commons/7/7e/Brennlinie.GIF', width=width))

(image by [User:Paul venter](https://commons.wikimedia.org/wiki/User:Paul_venter)).

Their study is essential in optics, as these are the bases for lenses and therefore of central importance in our image-centered society :-)

They are also of interest simply be cause they create mesmerizing patterns in nature, for in the pattern of wiggly lines on swimming pool bottoms, or on the sea floor at shallow depths. 

The formation of that specific pattern is very well illustrated by this animation (credit [Jacopo Bertolotti](https://twitter.com/j_bertolotti))

In [7]:
display(Image(url='https://upload.wikimedia.org/wikipedia/commons/2/2a/Caustics.gif', width=width))

What happens? Light travels fast (*very* fast) but it will travel slower in water than in the air. A consequence of this is that a ray changes slightly its direction as it hits the water. That is why a pen which is half immerged looks as if it is broken. In fact, it's only its image that gets distorted. 

The laws governing this deviations are well known as [Snell's law](https://en.wikipedia.org/wiki/Snell%27s_law) and state that

$$
\frac{\sin\theta_2}{\sin\theta_1} = \frac{n_1}{n_2}
$$
Where $n_1\approx 1$ and  $n_2\approx1.33$ are the refractive indeices of air and water, respectively.

Going back to the swimming pool, considering that the sun is emitting an uniform pattern of parallel rays, we may apply this lax to all light rays. The important ingredient is that the surface of the water (given as the surface height $z(x, y)$) changes smoothly and will deviate the light rays non-uniformly, thus creating the patterns. Writing down the equation, the displacement resulting from the angle of refraction is given by:

$$
\Delta x(x, y) = H \cdot \arcsin ( \frac 1 n \sin( \theta_I - \arctan(\frac{\partial z(x, y)}{\partial x}) + \arctan(\frac{\partial z(x, y)}{\partial x})
$$

Where $H$ is the pool's depth, $n$ is the refractive index and $\theta_I$ the incident angle.
The same formula applies in the other spatial dimension $y$ by computing the angle of the surface along this dimension as as $arctan(\frac{\partial z(x, y)}{\partial y})$. 

In the limit of a zenithal light and small angles, the equations simplifies a lot:
$$
\Delta x(x, y) \approx H' \cdot \frac{\partial z(x, y)}{\partial x}
$$
and
$$
\Delta y(x, y) \approx H' \cdot \frac{\partial z(x, y)}{\partial y}
$$

What is interesting here is that we will transform an incoming image (here uniform) by a set of small deformations of the support of the image. The equations are ultra-simple, but the consequences will paradoxically look complex.
Thinking about images as densities, it is very easy to simulate the transformation:

In [8]:
class Caustique:
    def __init__(self, opt):
        self.ratio = opt.ny/opt.nx # ratio between height and width (>1 for portrait, <1 for landscape)
        X = np.linspace(0, 1, opt.nx, endpoint=False) # vertical
        Y = np.linspace(0, self.ratio, opt.ny, endpoint=False) # horizontal
        self.xv, self.yv = np.meshgrid(X, Y, indexing='ij')
        self.opt = opt
        # https://stackoverflow.com/questions/16878315/what-is-the-right-way-to-treat-python-argparse-namespace-as-a-dictionary
        self.d = vars(opt)

    def wave(self):
        # A simplistic model of a wave using https://github.com/NeuralEnsemble/MotionClouds
        import MotionClouds as mc
        fx, fy, ft = mc.get_grids(self.opt.nx, self.opt.ny, self.opt.nframe)
        env = mc.envelope_gabor(fx, fy, ft, V_X=self.opt.V_Y, V_Y=self.opt.V_X, B_V=self.opt.B_V,
                                sf_0=self.opt.sf_0, B_sf=self.opt.B_sf, theta=self.opt.theta, B_theta=self.opt.B_theta)
        z = mc.rectif(mc.random_cloud(env, seed=self.opt.seed))
        return z

    def transform(self, z_):
        xv, yv = self.xv.copy(), self.yv.copy()
        
        dzdx = z_ - np.roll(z_, 1, axis=0)
        dzdy = z_ - np.roll(z_, 1, axis=1)
        xv = xv + self.opt.H * dzdx
        yv = yv + self.opt.H * dzdy

        xv = np.mod(xv, 1)
        yv = np.mod(yv, self.ratio)

        return xv, yv

    def plot(self, z, gifname=None, dpi=150):
        if gifname is None:
            os.makedirs(self.opt.tag, exist_ok=True)
            os.makedirs(f'/tmp/{self.opt.tag}', exist_ok=True)
            gifname=f'{self.opt.tag}/{self.opt.tag}.gif'
        binsx, binsy = self.opt.nx//self.opt.bin_dens, self.opt.ny//self.opt.bin_dens
        # edge_x, edge_y = np.linspace(0, 1, binsx+1, endpoint=True), np.linspace(0, self.ratio, binsy+1, endpoint=True)
        hist = np.zeros((binsx, binsy, self.opt.nframe))
        for i_frame in range(self.opt.nframe):
            xv, yv = self.transform(z[:, :, i_frame])
            hist[:, :, i_frame], edge_x, edge_y = np.histogram2d(xv.ravel(), yv.ravel(), 
                                                                 bins=[binsx, binsy], range=[[0, 1], [0, self.ratio]], density=True)

        hist /= hist.max()
        subplotpars = matplotlib.figure.SubplotParams(left=0., right=1., bottom=0., top=1., wspace=0., hspace=0.,)
        fnames = []
        for i_frame in range(self.opt.nframe):
            fig, ax = plt.subplots(figsize=(binsy/dpi, binsx/dpi), subplotpars=subplotpars)
            ax.pcolormesh(edge_y, edge_x, hist[:, :, i_frame], vmin=0, vmax=1, cmap=plt.cm.Blues_r)
            fname = f'/tmp/{gifname}_frame_{i_frame}.png'
            fig.savefig(fname, dpi=dpi)
            fnames.append(fname)
            plt.close('all')

        return make_gif(gifname, fnames, fps=self.opt.fps)


# a first instance

Let's generate a first instance of this caustic with the default parameters. Let's first have a look at the (simplistic) waveform we are using:

In [9]:
gifname = f'{opt.tag}/{opt.tag}.gif'
if not os.path.isfile(gifname):
    c = Caustique(opt)
    z = c.wave()
    gifname = c.plot(z)

It has many desirable properties: it is smooth in time and space, it is stochastic and periodic (no border effect). As such we may apply our simple transform and as a result we get:

In [10]:
if not os.path.isfile(f'{opt.tag}/{opt.tag}.{mc.vext}'):
    c = Caustique(opt)
    z = c.wave()
    mc.anim_save(z.swapaxes(0, 1), f'{opt.tag}/{opt.tag}')
mc.in_show_video(f'{opt.tag}', figpath=f'{opt.tag}')
display(Image(url=gifname, width=width))

""


Which ressembles what we expected to find.

## exploring parameters

The first thing to do is to explore is the effect of the pool's depth:

In [11]:
N_scan = 9
base = 2

In [ ]:
opt = init()
opt.tag = tag

c = Caustique(opt)
z = None
for H_ in c.opt.H*np.logspace(-1, 1, N_scan, base=base):
    print(f'H = {H_:.3f}')
    c.opt.H = H_
    gifname=f'{opt.tag}/{opt.tag}_H={H_:.3f}.gif'
    if not os.path.isfile(gifname):
        if z is None:
            z = c.wave()
        url=c.plot(z, gifname=gifname)
    display(Image(url=gifname, width=width))

H = 40.000


H = 47.568


This shows that for this waveform, there is an optimal depth to generate more concentrated caustics. This corresponds to the depth where the curvature of the waveform concentrates light as a lens. 

We may now explore the parameters of this waveform to see their effect on the resulting caustic:

In [ ]:
opt = init()

for variable in ['sf_0', 'B_sf', 'theta', 'B_theta', 'V_X', 'V_Y', 'B_V']:
    print(f'======{variable}======')
    for modul in np.logspace(-1, 1, N_scan, base=base):
        opt = init()
        opt.tag = tag

        c = Caustique(opt)
        c.d[variable] *= modul

        print(f'{variable}={variable}(default)*{modul:.3f}={c.d[variable]:.3E}')
        gifname = f'{opt.tag}/{opt.tag}_{variable}*={modul:.3f}.gif'
        if not os.path.isfile(gifname):
            z = c.wave()
            if not os.path.isfile(f'{opt.tag}/{opt.tag}_{variable}*={modul:.3f}.{mc.vext}'): 
                mc.anim_save(z.swapaxes(0, 1), f'{opt.tag}/{opt.tag}_{variable}*={modul:.3f}')
            url=c.plot(z, gifname=gifname)
        display(Image(url=gifname, width=width))

Future developments should:

* check if the approximation of small angles qualitatively changes the resulting images,
* use a more realistic waveform such as in https://laurentperrinet.github.io/sciblog/posts/2016-04-24_a-wave-going-backwards.html
* use different indices for different wavelengths,
* make it live (using pyglet for instance)

This happens in https://github.com/NaturalPatterns/2020_caustiques - I am happy to welcome any people that would be interested in exploring this.